# GNN Optimizer
This notebook runs the optimizer for the GNN model. With the use of Weights & Biases the optimal parameters are determined. Weights & Biases runs sweeps with different parameters while tracking the validation loss. With the data obtained from the runs, it can be determined which parameters play the most important roles and what their optimum values are.

In [ ]:
pip install torch_geometric==2.4.0

In [ ]:
import pickle
from sklearn.metrics import r2_score
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.optim as optim

from torch_geometric.data import Data
import torch.nn.functional as F
import torch_geometric as pyg
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import GCNConv, TAGConv, ChebConv
import networkx as nx
from sklearn.metrics import confusion_matrix

import timeit
start = timeit.default_timer()

## Data pre-processing
Here the train and validation data is loaded and normalized. Since the effect of the parameters on the validation loss plays the main roll, the test datasets are not used here.

In [ ]:
folder_path = 'FLOOD/GNN_model/model'
%cd "$folder_path"
import GNN_model as gnn

folder_path = 'FLOOD/GNN_model/processing'
%cd "$folder_path"
import GNN_preprocessing as pre

In [ ]:
G = nx.grid_2d_graph(64, 64, create_using=nx.DiGraph)
pos = {i:(x+0.5,y+0.5) for i, (x,y) in enumerate(G.nodes())} # Position mapping for grid layout
mapping = dict(zip(G, range(0, G.number_of_nodes())))
G = nx.relabel_nodes(G, mapping)

In [ ]:
folder_path = 'FLOOD/raw_dataset'
%cd "$folder_path"

# importing the training set (80 simulations) and adding them to graphs
DEMS, WDS = pre.load_data(1, 80, folder_path, augmentation=True, augmentation_per=1)

# Choose the same random state, so DEM and WD are splitted the same way, 30% for validation
train_dataset_DEM, val_dataset_DEM = train_test_split(DEMS, test_size=0.3, random_state=42)
train_dataset_WD, val_dataset_WD = train_test_split(WDS, test_size=0.3, random_state=42)

In [ ]:
# normalizing the DEMs
DEM_scaler = pre.CustomMinMaxScaler_interpolation((0,1)) # custom minmax scaler
DEM_scaler.fit(train_dataset_DEM)

scaled_DEM_train = DEM_scaler.transform(np.array(train_dataset_DEM))
scaled_DEM_val = DEM_scaler.transform(np.array(val_dataset_DEM))

# normalizing the WDs
WD_scaler = pre.CustomMinMaxScaler_interpolation((0,1)) # custom minmax scaler

WD_scaler, train_dataset = pre.normalize_WD(WD_scaler, train_dataset_WD, scaled_DEM_train, G, pos, train=True)
val_dataset = pre.normalize_WD(WD_scaler, val_dataset_WD, scaled_DEM_val, G, pos)

## Log in into Weights & Biases
Note that to run this notebook, you need a Weights & Biases account.

In [ ]:
!pip install wandb
import wandb

In [ ]:
wandb.login()

## Define sweep
The sweep method chosen is 'random', to make sure many different combinations are tried. This then clearly shows which parameters are more important and what their optimal values are.

Note a combination of high K, high batch size and high number of MLP layers leads to a crash (ran out of CUDA memory). The follow-up runs will then also crash, even the combination of parameters is less computationally expensive. Therefore it is recommended to test the highest computational expensive run first in the model notebook, before running the optimization notebook. It is does not crash in the model notebook, this notebook will also safely run.

In [ ]:
# Choose method for configurations
sweep_config = {
    'method': 'random'
    }

# Choose metric for optimization
metric = {
    'name': 'val_loss',
    'goal': 'minimize'
    }
sweep_config['metric'] = metric

# Choose parameters to vary
parameters_dict = {
    'n_layers_MLP': {
        'values': [2, 3, 4]
        },
    'n_layers_GNN': {
        'values': [3, 4, 6, 8]
        },
    'K_tot': {
        'values': [96, 120, 144, 168, 192]
        },
   'hidden_features': {
        'values': [64, 128]
        },
    'convolution_type': {
        'values': ['ChebConv', 'TAGConv', 'GCNConv']
        },
    'batch_size': {
        'values': [8, 16]
        }
    }

sweep_config['parameters'] = parameters_dict

# Define parameters which are not altered during the sweep, but still want to be mentioned
parameters_dict.update({
    'epochs': {
        'value': 200}
    })
print(sweep_config)

In [ ]:
# Define sweep id variable
sweep_id = wandb.sweep(sweep_config, project='encoder_decoder_GNN')

## Run the Sweep agent

In [ ]:
def train_epoch(model, loader, optimizer, loss_function, device='cpu'):
    model.to(device)
    model.train()
    losses = []

    for batch in loader:
        batch = batch.to(device)
        preds = model(batch)
        loss = loss_function(preds, batch.y)

        losses.append(loss.cpu().detach())

        loss.backward()   # compute the gradients using backpropagation
        optimizer.step()  # update the weights with the optimizer
        optimizer.zero_grad(set_to_none=True)   # reset the computed gradients

    return np.array(losses).mean()

def evaluation(model, loader, loss_function, device='cpu'):
    model.to(device)
    model.eval() # specifies that the model is in evaluation mode
    losses = []

    # Remove gradients computations since we are only evaluating and not training
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            preds = model(batch)

            loss = loss_function(preds, batch.y)
            losses.append(loss.cpu())

    return np.array(losses).mean()

# Set training parameters
learning_rate = 0.001

# Create loss function
loss_function = nn.MSELoss()

# This line is used to select GPU to train, if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(config=None):
    with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
    config = wandb.config
    K = config.K_tot // config.n_layers_GNN
    network = gnn.build_network(config.hidden_features, config.n_layers_MLP, config.n_layers_GNN, K, config.convolution_type).to(device)
    optimizer = torch.optim.AdamW(network.parameters(), lr=learning_rate)
    batch_size = config.batch_size

    # Create the training and validation dataloaders to "feed" data to the model in batches
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    best_val = np.inf # start with an infinite high validation loss, so it will be updated with the first computed loss
    best_epoch = 0
    for epoch in range(config.epochs):
        train_loss = train_epoch(network, train_loader, optimizer, loss_function, device=device)
        val_loss = evaluation(network, val_loader, loss_function, device=device)
        if val_loss < best_val: # update the best validation loss
        best_val = val_loss
        best_epoch = epoch
        if epoch - best_epoch > 20: # stop if the validation loss was not improved for more than 20 epochs
        break

    wandb.log({'val_loss': best_val, 'best val at epoch': best_epoch})

In [ ]:
# Start the sweep, count tells how many configurations are going to be done
wandb.agent(sweep_id, function=train, count=100)